### Name: Hung Pham
### Email address: mryo.hp@gmail.comn

####  <u>References and sources </u>

https://www.tensorflow.org/tutorials/load_data/images

In [ ]:
!pip install -q tfds-nightly tensorflow matplotlib

#### Load libraries and set random number generator seed

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.metrics import confusion_matrix
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [ ]:
np.random.seed(42)

In [ ]:
# malaria

In [ ]:
(ds_train, ds_validation, ds_test), ds_info = tfds.load('malaria',
                                      split=['train[:70%]', 'train[70%:85%]', 'train[85%:]'],
                                      shuffle_files=True, as_supervised=True, with_info=True)

In [ ]:
name = ds_info.features['label'].names
num_classes = ds_info.features['label'].num_classes
labels = ds_info.features["label"]
print("Target Classes: "+str(name))
print("Num classes: "+ str(num_classes))

In [ ]:
def get_label(num):
    return labels.int2str(num)

In [ ]:
print(ds_info)

In [ ]:
tfds.visualization.show_examples(ds_train, ds_info)

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = 130
# LEARNING_RATE = 0.001

print("Training size: " + str(len(ds_train)))
print("Validation set size: " + str(len(ds_validation)))
print("Testing set size: " + str(len(ds_test)))

In [ ]:
def resize(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize_with_crop_or_pad(image, IMG_SIZE, IMG_SIZE)
    return image, label

In [ ]:
for image, label in ds_train.take(2):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())

In [ ]:
ds_train_resized = (
    ds_train
    .cache()
    .map(resize)
    .batch(BATCH_SIZE)
) 

ds_validation_resized = (
    ds_validation
    .cache()
    .map(resize)
    .batch(BATCH_SIZE)
) 
ds_test_resized = (
    ds_test
    .cache()
    .map(resize)
    .batch(BATCH_SIZE)
) 

In [ ]:
image_batch, label_batch = next(iter(ds_train_resized))

def show_batch(image_batch, label_batch):
    plt.figure(figsize=(12,12))
    for n in range(25):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(image_batch[n])
        plt.title("{}".format(get_label(label_batch[n])))
        plt.axis("off")
show_batch(image_batch.numpy(), label_batch.numpy())

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
ds_train_resized = ds_train_resized.cache().prefetch(buffer_size=AUTOTUNE)
ds_validation_resized = ds_validation_resized.cache().prefetch(buffer_size=AUTOTUNE)
ds_test_resized = ds_test_resized.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
for images, lables in ds_train_resized:
    print(images.shape)
    break

In [ ]:
INPUT_SHAPE = (IMG_SIZE, IMG_SIZE, 3) 
model = tf.keras.Sequential([
    tf.keras.Input(shape=INPUT_SHAPE),
    tf.keras.layers.Conv2D(16, 3, activation='elu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='
    elu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])
model.summary()

In [ ]:
# loss_fn = keras.losses.SparseCategoricalCrossentropy()
# loss_fn = 'categorical_crossentropy'
loss_fn = 'binary_crossentropy'
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

In [ ]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("model.h5", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5,restore_best_weights=True)

In [ ]:
history = model.fit(
    ds_train_resized, epochs=20,
    validation_data=ds_validation_resized,
    callbacks=[checkpoint_cb]
)

In [ ]:
model.evaluate(ds_test_resized)

In [ ]:
from matplotlib.pyplot import figure

figure(figsize=(15, 12), dpi=60)
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()


In [ ]:
model.evaluate(ds_train_resized)
figure(figsize=(15, 12), dpi=60)
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [ ]:
model.evaluate(ds_test_resized)

In [ ]:
new_model = tf.keras.models.load_model('model.h5')
new_model.summary()
new_model.evaluate(ds_test_resized)